# Calculate Accuracy for Mistral/textminr_ner
by Andreas Sünder & Benjamin Kissinger

## Load the Base Model

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Load Finetuned Model

In [2]:
from peft import PeftModel
ft_model = PeftModel.from_pretrained(base_model, "textminr/ner-mistral-7b")

## Load Dataset

In [12]:
from datasets import load_dataset
dataset = load_dataset('json', data_files="../augmentation/data/test.jsonl", split='train')
amount_examples = dataset.num_rows
amount_correct = 0

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

## Calculate Accuracy

In [13]:
from tqdm import tqdm

answer = ""
prompt_template = 'Input sentence: {} --- Output:'

ft_model.eval()
for row in tqdm(dataset):
    if len(row["prompt"]) > 140:
      model_input = tokenizer(prompt_template.format(row["prompt"]), return_tensors="pt").to("cuda")
    else:
      model_input = tokenizer(prompt_template.format(row["prompt"]+row["prompt"]), return_tensors="pt").to("cuda")
    with torch.no_grad():
        x = tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=150)[0], skip_special_tokens=True)
        if row["response"] in x:
            print("Correct: ", row["response"], x)
            amount_correct += 1
        else:
            print("Incorrect: ", row["response"], x)

print("Accuracy: " + str(amount_correct / amount_examples))

  2%|▏         | 1/52 [00:01<01:28,  1.73s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Correct:  {"author": "Isabella Thompson", "date": "2015"} Input sentence: Exploring the wonders of ancient civilizations, 'Timeless Echoes' by Isabella Thompson took center stage in Robert's life in the year 2015.Exploring the wonders of ancient civilizations, 'Timeless Echoes' by Isabella Thompson took center stage in Robert's life in the year 2015. --- Output: {"author": "Isabella Thompson", "date": "2015"}


  4%|▍         | 2/52 [00:03<01:26,  1.74s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Correct:  {"author": "Luna Starlight", "date": "2020"} Input sentence: Amidst the cosmic ballet, Astral Voyager Orion, captivated by 'Celestial Harmonies' penned by Luna Starlight in the year 2020, embarks on an interstellar odyssey. --- Output: {"author": "Luna Starlight", "date": "2020"}


  6%|▌         | 3/52 [00:05<01:20,  1.65s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Incorrect:  {"author": "Benjamin Robinson", "date": "2017"} Input sentence: In 2019, during a season of reflection, Daniel Robinson discovered 'Uncharted Realms,' authored by Benjamin Robinson in 2017. Sophia finally explored its realms in 2020. Benjamin's insights sparked a transformative journey for Sophia, shaping her perspectives and leading to the abandonment of previous pursuits. --- Output: {"author": "Daniel Robinson", "date": "2019"}


 98%|█████████▊| 51/52 [01:31<00:01,  1.66s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Correct:  {"author": "N/A", "date": "2015"} Input sentence: A short story crafted on a winter evening in 2015, reflecting on the joy of childhood.A short story crafted on a winter evening in 2015, reflecting on the joy of childhood. --- Output: {"author": "N/A", "date": "2015"}


100%|██████████| 52/52 [01:33<00:00,  1.79s/it]

Incorrect:  {"author": "Emily Turner", "date": "2019"} Input sentence: Amidst the challenges of 2021, Abigail Turner found encouragement in 'Serenity Unveiled,' written by Emily Turner in 2019. James immersed himself in its pages in 2022. Emily's perspectives became a driving force for James, shaping his goals and prompting him to let go of past hobbies. --- Output: {"author": "Abigail Turner", "date": "2021"}
Accuracy: 0.36538461538461536
